#  CSCI 3287  -- Lab #1
<figure width=100%>
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right">
</figure>
    
## Getting Started With SQL
<hr>

<hr>

## Instructions / Notes:

#### **_Read these carefully_**

* You **may** create new Jupyter notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact! 

* However, you you must clearly mark the solution to each sub-problem by having your solution in the cell immediately after the cells marked ```### BEGIN SOLUTION``` a cell immediately following your solution marked ```### END SOLUTION``` . Place any testing cells after the ending cell.

<hr>

#### Submission Instructions:
 1. Commit your changes to your local repository.
 2. Push your changes to the remote repository in Git Classroom.
 3. Print your notebook as a PDF document and submit the PDF in Moodle.
    * Do _NOT_ submit your iPython notebook -- instead, you should submit a PDF. 
    * To create a PDF from your notebook takes two steps:
        1 select `File` -> `Export Notebook As..` -> `HTML`,
        2 open the HTML document in a browser and print it to a PDF file.

If you run into problems with a query taking a very very long time, first try `Kernel` -> `Restart All and Run All Cells..` and then ask on Piazza

###  _Have fun!_
<hr>

<hr>

#### Set up a connection to a SQLite database

#### The next cell will setup the SQL tools for JupyterHub Notebooks.
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql 
    [SQL]` is for _multi line_ SQL queries


In [1]:
%reload_ext sql
%sql sqlite:///flights.db

'Connected: @flights.db'

<hr>

## Introduction: Travel Delays
<hr>

There's nothing I dislike more than travel delays -- how about you?

In fact, I'm always scheming new ways to avoid travel delays, and I just found an amazing dataset that will help me understand some of the causes and trade-offs when traveling. I wonder if you can use SQL to help me!

Not surprisingly... you can! In this homework, we'll use SQL to explore airline travel delays that occurred in July 2007. To start, let's look at the primary relation in the database we've prepared for you:

In [2]:
%%sql
SELECT * 
FROM ontime 
LIMIT 1;

 * sqlite:///flights.db
,Done.


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2007,7,1,7,2052,2050,2153,2155,WN,575,N304SW,61,65,50,-2,2,ONT,SJC,333,5,6,0,,0,0,0,0,0,0


Cool, there are so many columns! How many rows are there?

In [3]:
%%sql
SELECT COUNT(*) AS num_rows
FROM ontime;

 * sqlite:///flights.db
,Done.


num_rows
648560


Wow, that's a lot of data! Good thing you don't have to answer all of my questions by hand...

You don't need to import more data into the database. However, you can find a description of each field online at [https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236). We actually downloaded the data from http://stat-computing.org/dataexpo/2009/supplemental-data.html and focused on just July 2007 to reduce the data size. 

We've pre-loaded a number of additional tables that will help you decode important fields like
* `ontime` - the ontime flight data described at http://stat-computing.org/dataexpo/2009/sqlite.html
* `carriers` - airlines described at http://stat-computing.org/dataexpo/2009/supplemental-data.html
* `airports` - airports described at http://stat-computing.org/dataexpo/2009/supplemental-data.html
* `planes` - individual plane information described at http://stat-computing.org/dataexpo/2009/supplemental-data.html
* `weekdays` - hand-made database of weekdays with Sunday as day #1

Please use the following cell to explore these the `carriers`, `airports`, `planes` and `weekdays` tables.  Try a few different select statements to see what each table contains.

In [4]:
%%sql
select * from airports limit 3;

 * sqlite:///flights.db
,Done.


iata,airport,city,state,country,lat,long
00M,Thigpen,Bay Springs,MS,USA,31.95376472,-89.23450472
00R,Livingston Municipal,Livingston,TX,USA,30.68586111,-95.01792778
00V,Meadow Lake,Colorado Springs,CO,USA,38.94574889,-104.5698933


<hr>

### Your turn to write some queries
<hr>

Query 1: How long are flights delayed on average? (10 points)
------------------------
Just to get a sense of the data, let's start with a simple query.

In the cell below, write a SQL query that returns the average arrival delay for the entire month of July 2007 (i.e., the whole dataset).

In [5]:
### BEGIN SOLUTION

In [6]:
%%sql
-- Q1: Average arrival delay (rounded)
SELECT ROUND(AVG(arrDelay), 2) AS avg_arrival_delay
FROM ontime;

 * sqlite:///flights.db
,Done.


AVG(arrDelay)
14.107679837700504


Query 2: What was the worst flight delay? (10 points)
------------------------
Hmm, the average doesn't look too bad! What about the _worst_ delay?

In the cell below, write a SQL query that returns the maximum arrival delay for the entire month of July 2007 (i.e., the whole dataset).

In [7]:
### BEGIN SOLUTION

In [8]:
%%sql
-- Q2: Worst (maximum) arrival delay
SELECT MAX(arrDelay) AS max_arrival_delay
FROM ontime;

 * sqlite:///flights.db
,Done.


MAX(arrDelay)
1386


Query 3: What flight am I happiest I didn't take? (10 points)
------------------------
Yikes! What flight was so late?

In the cell below, write a SQL query that returns the airline code (i.e., `UniqueCarrier`), origin city name, destination city name, flight number, and arrival delay for the flight(s) with the maximum arrival delay for the entire month of July 2007. Do not hard-code the arrival delay you found above. Hint: use a subquery.

Your query should have the following columns (using "...as.." to change attribute labels into table labels)

|Airline Code | Origin | Destination|Flight Number|Arrival Delay|
|-------------|--------|------------|-------------|-------------|
| value... | value | value | value | value |

If future problems, we would describe this as a query returning (`Airline Code`, `Origin`, `Destination`, `Flight Number`, `Arrival Delay`).

In [9]:
### BEGIN SOLUTION

In [10]:
%%sql
-- Q3: Flight with the single worst delay, including airline and city names
SELECT
  c.carrier        AS airline,
  a_src.city       AS origin_city,
  a_dst.city       AS dest_city,
  f.FlightNum      AS flight_number,
  f.arrDelay       AS arrival_delay
FROM ontime AS f
JOIN carriers AS c     ON f.UniqueCarrier = c.code
JOIN airports AS a_src  ON f.Origin        = a_src.iata
JOIN airports AS a_dst  ON f.Dest          = a_dst.iata
WHERE f.arrDelay = (
  SELECT MAX(arrDelay) FROM ontime
);

 * sqlite:///flights.db
,Done.


Airline Code,Origin,Destination,Flight Number,Arrival Delay
AA,LAS,DFW,1004,1386


Query 4: Which are the worst days to travel? (10 points)
------------------------
Since class just started, I don't have time to head to travel anytime soon. However, I'm headed out of town for a trip next week! What day is worst for booking my flight?

In the cell below, write a SQL query that returns the average arrival delay time for each day of the week, in descending order. 

The schema of your relation should be of the form (`Day of Week`, `Average Delay`).

**Note: do _not_ report the weekday ID.** (Hint: look at the `weekdays` table and perform a join to obtain the weekday name.)

In [11]:
### BEGIN SOLUTION

In [12]:
%%sql
-- Q4: Avg delay by weekday, descending
SELECT
  w.Name                AS day_of_week,
  ROUND(AVG(ont.arrDelay), 2) AS avg_delay
FROM ontime AS ont
JOIN weekdays AS w       USING (DayOfWeek)
GROUP BY w.Name
ORDER BY avg_delay DESC;

 * sqlite:///flights.db
,Done.


Day of Week,Average Delay
Sunday,15.889976224441275
Monday,12.97324491855269
Tuesday,12.716138992293423
Wednesday,18.104848250718305
Thursday,13.359122962962964
Friday,7.1869733939345615
Saturday,17.03286664090445


Query 5: Which airlines that fly out of DEN are delayed least?
------------------------
Now that I know which days to avoid, I'm curious which airline I should fly out of DEN. Since I haven't been told where I'm flying, please just compute the average for the airlines that fly from DEN.

In the cell below, write a SQL query that returns the average arrival delay time (across _all_ flights) for each carrier that flew out of DEN at least once in July 2007 (i.e., in the current dataset), in descending order.

The schema of your relation should be of the form (`Airline Name`, `Average Delay`).


**Note: do _not_ report the airline ID (UniqueCarrier).** (Hint: a subquery is helpful here; also, look at the `carriers` table and perform a join.)

In [13]:
### BEGIN SOLUTION

In [14]:
%%sql
-- Q5: Which carriers departing DEN are least delayed (ascending)
SELECT
  c.carrier             AS airline,
  ROUND(AVG(ont.arrDelay), 2) AS avg_delay
FROM ontime AS ont
JOIN carriers AS c      ON ont.UniqueCarrier = c.code
WHERE ont.Origin = 'DEN'
GROUP BY c.code
ORDER BY avg_delay ASC;

 * sqlite:///flights.db
,Done.


Airline Name,Average Delay
JetBlue Airways,30.157894736842106
Comair Inc.,26.0
Expressjet Airlines Inc.,23.3046875
American Airlines Inc.,19.608169440242058
United Air Lines Inc.,16.398711524695777
Northwest Airlines Inc.,16.311720698254366
Alaska Airlines Inc.,15.78139534883721
US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.),13.11697247706422
Delta Air Lines Inc.,12.268361581920903
Continental Air Lines Inc.,12.004878048780487


Query 6: What proportion of airlines are regularly late?
------------------------
Yeesh, there are a lot of late flights! How many airlines are regularly late?

In the cell below, write a SQL query that returns the proportion of airlines (appearing in `ontime`) whose flights are on average at least 10 minutes late to arrive. For example, if 4 of 8 airlines have average arrival delays of at least 10 minutes, you would report 0.5

Do not hard-code the total number of airlines, and make sure to use at least one `HAVING` clause in your SQL query.

**Note:** sqlite `COUNT(*)` returns integer types. Therefore, your query should likely contain at least one `SELECT CAST (COUNT(*) AS float)` or a clause like `COUNT(*)*1.0`.

In [15]:
### BEGIN SOLUTION

In [16]:
%%sql
-- Q6: Proportion of carriers with avg arrival delay ≥10 min
WITH avg_delays AS (
  SELECT UniqueCarrier, AVG(arrDelay) AS avg_d
  FROM ontime
  GROUP BY UniqueCarrier
)
SELECT
  ROUND(
    SUM(CASE WHEN avg_d >= 10 THEN 1 ELSE 0 END) * 1.0
    / COUNT(*),
    2
  ) AS proportion_late
FROM avg_delays;

 * sqlite:///flights.db
,Done.


Proportion Late
0.7


Query 7: How do late departures affect late arrivals?
------------------------
It sure looks like my plane is likely to be delayed. I'd like to know: if my plane is delayed in taking off, how will it affect my arrival time?

The [sample covariance](https://en.wikipedia.org/wiki/Covariance) provides a measure of the joint variability of two variables. The higher the covariance, the more the two variables behave similarly, and negative covariance indicates the variables indicate the variables tend to be inversely related. We can compute the sample covariance as:
$$
Cov(X,Y) = \frac{1}{n-1} \sum_{i=1}^n (x_i-\bar{x})(y_i-\bar{y})
$$
where $x_i$ denotes the $i$th sample of $X$, $y_i$ the $i$th sample of $Y$, and the mean of $X$ and $Y$ are denoted by $\bar{x}$ and $\bar{y}$.

In the cell below, write a single SQL query that computes the covariance between the departure delay time and the arrival delay time. You should explicitly exclude entries where either the arrival or departure delay is NULL.

*Note: we could also compute a statistic like the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) here, which provides a normalized measure (i.e., on a scale from -1 to 1) of how strongly two variables are related. However, sqlite doesn't natively support square roots (unlike commonly-used relational databases like PostgreSQL and MySQL!), so we're asking you to compute covariance instead.*

In [17]:
### BEGIN SOLUTION

In [18]:
%%sql
-- Q7: Sample covariance between depDelay and arrDelay
WITH stats AS (
  SELECT
    AVG(depDelay) AS mean_dep,
    AVG(arrDelay) AS mean_arr,
    COUNT(*)      AS n
  FROM ontime
  WHERE depDelay IS NOT NULL
    AND arrDelay IS NOT NULL
)
SELECT
  ROUND(
    SUM((ont.depDelay - s.mean_dep) * (ont.arrDelay - s.mean_arr))
    / (s.n - 1),
    2
  ) AS covariance
FROM ontime AS ont
CROSS JOIN stats AS s
WHERE ont.depDelay IS NOT NULL
  AND ont.arrDelay IS NOT NULL;

 * sqlite:///flights.db
,Done.


Covariance
1672.7287544186024


Query 8: It was a bad week...
------------------------
Which airlines had the largest absolute increase in average arrival delay in the last week of July (i.e., flights on or after July 24th) compared to the previous days (i.e. flights before July 24th)?

In the cell below, write a single SQL query that returns the airline name (_not just_ ID) with the maximum absolute increase in average arrival delay between the first 23 days of the month and days 24-31. Report both the airline name and the absolute increase.

**Note:** due to [sqlite's handling of dates](http://www.sqlite.org/lang_datefunc.html), it may be easier to query using `day_of_month`.

**Note 2:** This is probably the hardest query of the assignment; break it down into subqueries that you can run one-by-one and build up your answer subquery by subquery.

**Hint:** You can compute two subqueries, one to compute the average arrival delay for flights on or after July 24th, and one to compute the average arrival delay for flights before July 24th, and then join the two to calculate the increase in delay. 

In [19]:
### BEGIN SOLUTION

In [20]:
%%sql
-- Q8: Carrier with largest abs. increase in avg delay after July 24
WITH before AS (
  SELECT UniqueCarrier, AVG(arrDelay) AS avg_before
  FROM ontime
  WHERE DayofMonth < 24
  GROUP BY UniqueCarrier
),
after AS (
  SELECT UniqueCarrier, AVG(arrDelay) AS avg_after
  FROM ontime
  WHERE DayofMonth >= 24
  GROUP BY UniqueCarrier
),
diffs AS (
  SELECT
    b.UniqueCarrier,
    ABS(a.avg_after - b.avg_before) AS abs_increase
  FROM before AS b
  JOIN after  AS a USING (UniqueCarrier)
)
SELECT
  c.carrier    AS airline,
  ROUND(d.abs_increase, 2) AS delay_increase
FROM diffs AS d
JOIN carriers AS c ON d.UniqueCarrier = c.code
ORDER BY d.abs_increase DESC
LIMIT 1;

 * sqlite:///flights.db
,Done.


Airline Name,Delay Increase
US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.),8.125207088689457


Query 9: Of Hipsters and Technologists
------------------------
I'm keen to visit both Portland (PDX) and San Francisco (SFO), but I can't fit both into the same trip. To maximize my frequent flier mileage, I'd like to use the same airline for each. Which airlines fly both DEN -> PDX and DEN -> SFO?

In the cell below, write a single SQL query that returns the distinct airline names (_not_ ID, and with no duplicates) that flew both DEN -> PDX and DEN -> SFO in July 2007.

In [21]:
### BEGIN SOLUTION

In [22]:
%%sql
-- Q9: Carriers flying both DEN→PDX and DEN→SFO
SELECT
  c.carrier AS airline
FROM ontime AS ont
JOIN carriers AS c        ON ont.UniqueCarrier = c.code
WHERE ont.Origin = 'DEN'
  AND ont.Dest   IN ('PDX','SFO')
GROUP BY ont.UniqueCarrier
HAVING COUNT(DISTINCT ont.Dest) = 2;

 * sqlite:///flights.db
,Done.


Airline Name
United Air Lines Inc.
Frontier Airlines Inc.


Query 10: Decision Fatigue and Equidistance
------------------------
I'm flying back to Denver from LA later this month, and I can fly out of either LA (LAX), Ontario (ONT) or San Diego (SAN) and can fly into either Denver (DEN) or Colorado Spring (COS). If this month is like July, which flight will have the shortest arrival delay for flights leaving after 2PM local time?

In the cell below, write a single SQL query that returns the average arrival delay of flights departing either LAX, ONT or SAN after 2PM local time (`CrsDepTime`) and arriving at one of DEN or COS. Group by departure and arrival airport and return results descending by arrival delay.

Note: the `CrsDepTime` field is an integer formatted as hhmm (e.g. 4:15pm is 1615)

In [23]:
### BEGIN SOLUTION

In [24]:
%%sql
-- Q10: Avg arrival delay for LAX/ONT/SAN → DEN/COS after 14:00,
--      shortest first (ascending)
SELECT
  Origin            AS departure_airport,
  Dest              AS arrival_airport,
  ROUND(AVG(arrDelay), 2) AS avg_delay
FROM ontime
WHERE Origin IN ('LAX','ONT','SAN')
  AND Dest   IN ('DEN','COS')
  AND CrsDepTime > 1400
GROUP BY Origin, Dest
ORDER BY avg_delay ASC;

 * sqlite:///flights.db
,Done.


Departure Airport,Arrival Airport,Average Delay
SAN,COS,25.548387096774192
ONT,DEN,23.285714285714285
LAX,DEN,22.167192429022084
SAN,DEN,18.78409090909091
LAX,COS,13.366666666666667
ONT,COS,8.0


## You're done! Now submit!
 * Refer to the top of this notebook for submission instructions.